In [ ]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
import os

In [ ]:
def latestNews(topic):
    # in reality this would actually search up a news article, but rn it's just a random fixed article
    news = "China is building its first port dedicated to deep-sea research in the South China Sea as the country seeks to cement its claims over disputed waters in the area. The project broke ground last week in the resort city of Sanya on the island province of Hainan. The China State Shipbuilding Corporation, a state-owned ship-maker, said it would serve the interests of national defence as well as supporting deep-sea experiments. The port will be the core of the research base, docking experimental vessels and ensuring “highly efficient operations”, the state-owned company said."
    end = "\nPlease summarise this news for the user as a final answer."

    return "The latest news on " + topic + " is " + news + end

def defenseInformation(string):
    string = string.lower()
    if string == "army":
        return """"The Singapore Army is the land service branch of the Singapore Armed Forces (SAF). The largest of the four branches of the SAF, the Singapore Army traces its origins to the 1st Battalion, Singapore Infantry Regiment (1 SIR), which was formed in 1957, when Singapore was still under British colonial rule. After Singapore's independence on 9 August 1965, the Singapore Army Bill was passed in Parliament on 23 December 1965, and National Service (NS) was subsequently introduced in 1967. Mostly made up of conscripts, the Singapore Army can mobilise all operationally-ready military reservists in the event of war or national exigencies."""
    if string == "navy":
        return "The Republic of Singapore Navy (RSN) is the naval service branch of the Singapore Armed Forces (SAF) responsible for defending the country against any sea-borne threats, and the protection of its sea lines of communications, that would compromise Singapore as a global trading hub. The RSN traces its origins to the Royal Navy when Singapore was still a Crown colony of the British Empire. After Singapore's independence from Malaysia in 1965, the service was formally established in 1967, and had undergone a substantial modernisation ever since—which led them into becoming the most powerful navy in Southeast Asia."
    if string == "air forces":
        return "The Republic of Singapore Air Force (RSAF) is the aerial service branch of the Singapore Armed Forces (SAF) responsible for controlling and defending the airspace of the country, and providing air support to the Army and Navy. It was established in 1968 as the Singapore Air Defence Command (SADC) before renaming to its current name in 1975.[5] As the most powerful air force in Southeast Asia, the RSAF has undertaken a significant role in Singapore's military defence strategy since its formation."
    if string == "digital intelligence":
        return "The Digital and Intelligence Service (DIS) is the digital service branch of the Singapore Armed Forces (SAF) responsible for providing military intelligence to the armed forces, building up the country's digital defence capabilities, and protecting the psychological defence of its military personnel. It was established on 28 October 2022, in response to the increased number of attacks by non-state actors, and the resulting damage from the Russian-Ukrainian cyberwarfare."
    return "Invalid input"

def makeIncidentReport(report):
    # code that actually makes an incident report
    return "Incident report made successfully"

In [ ]:
tools = [
    Tool(
        name = "Latest News",
        func = latestNews,
        description="useful for when you need to get the latest news on any specified topic. The input to this tool should be the topic or category of news that should be retrieved. For example, a topic could be the Russia-Ukraine war.",
    ),
    Tool(
        name = "Defense Information",
        func=defenseInformation,
        description="useful for when you need to find background information about Singapore's defense force, including the army, navy, air force and the digital and intelligence service. The input to this tool should be one of [army, navy, air force, digital intelligence] depending on which branch you are interested in."
    ),
        Tool(
        name = "Make Incident Report",
        func=makeIncidentReport,
        description="useful for when the user asks for an incident report to be made. The input to this tool is the string that the user inputs."
    ),
]


In [ ]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Try using the chat history before using a tool, if possible. Remember to end with 'Final Answer: '. Begin!

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
# os.environ["OPENAI_API_KEY"] = ADD API KEY!!!

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo', max_retries=1)

In [ ]:
# Construct LLMChain with memory and agent
llm_chain = LLMChain(llm=llm, prompt=prompt)

agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True, handle_parsing_errors=True,)

agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [ ]:
print(agent.llm_chain.prompt.template)

## Running

In [ ]:
agent_chain.run(input = "how old is singapore's army in 2023?")

In [ ]:
agent_chain.run(input = "what about its navy?")

## Manipulating memory for testing purposes

In [ ]:
memory.load_memory_variables({})

In [ ]:
memory.save_context({"input": "what is going on in the South China Sea?"}, {"output": "The latest news on South China Sea is China is building its first port dedicated to deep-sea research in the South China Sea as the country seeks to cement its claims over disputed waters in the area. The project broke ground last week in the resort city of Sanya on the island province of Hainan. The China State Shipbuilding Corporation, a state-owned ship-maker, said it would serve the interests of national defence as well as supporting deep-sea experiments. The port will be the core of the research base, docking experimental vessels and ensuring “highly efficient operations”, the state-owned company said."})

In [ ]:
memory.clear()